In [22]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
import geopandas as gpd
from pathlib import Path
import pathlib
import os
import requests, zipfile, io
import plotly
from sklearn import linear_model

import warnings
warnings.filterwarnings('ignore')

Acknowledgement: Much of this code was helped made from my homework responses to AQI project from Data200 fall 2021. I give credit to the course staff for guiding me in many of these EDA ideas through the problem statements. Any code not provided in class was written by me

In [23]:
#works for windows!!
epa_data = {}
for root,dirs, files in os.walk(".",topdown=True):
    root_old = root
    p = pathlib.Path(root_old)
    root = p.as_posix()
#     print('root: '+root)
    for name in files:
#         print('name: '+name)
        if name[-4:] == ".csv":
            path = os.path.join(root_old, name)
            data = pd.read_csv(path)
#             print('path: '+path)
            if ".\\AQI_Data\\" in root_old:
                pattern = r".\\AQI_Data\\(\w+)\\\w+.csv"
                ls_match = re.findall(pattern,str(path))
                if "/.ipynb" not in root:
                    print(ls_match)
                    name = ls_match[0]
                    epa_data[name] = data
            else:
                epa_data[name[:-4]] = data

In [24]:
epa_data.keys()

dict_keys(['2020', 'daily_global_weather_2020', 'us_air_quality_measures', 'us_greenhouse_gas_emissions_direct_emitter_facilities', 'us_greenhouse_gas_emission_direct_emitter_gas_type', 'epa_filenames', 'Traffic_Volumes_AADT', 'annual_aqi_by_county_2020', 'aqs_sites', 'daily_42101_2020', 'daily_aqi_by_county_2020', 'daily_42602_2020', 'daily_44201_2020', 'daily_42401_2020', 'daily_TEMP_2020', 'daily_WIND_2020', 'epa_data_CA_merged'])

https://www.epa.gov/outdoor-air-quality-data/about-air-data-reports
Site ID:
   FIPS state code (2) - FIPS county code (3) - AQS site code (4)
e.g 10-003-2004

In [101]:
original_so2 = epa_data.get('daily_42401_2020')
so2 = original_so2.loc[original_so2['State Name'] == 'California']
so2 = so2.loc[:,['Date Local','Arithmetic Mean' ,'1st Max Value', 'AQI']].dropna()
regr = linear_model.LinearRegression()
regr.fit(so2[['Arithmetic Mean' ,'1st Max Value']], so2.loc[:,'AQI'])
print(np.corrcoef(so2.loc[:,'1st Max Value'], so2.loc[:,'AQI'])[0][1])
print(np.corrcoef(so2.loc[:,'Arithmetic Mean'], so2.loc[:,'AQI'])[0][1])
print(regr.intercept_)
regr.coef_

0.9677106173790913
0.6753062755116581
-0.33851259236973297


array([0.22342961, 1.1178662 ])

In [102]:
orgininal_no2 = epa_data.get('daily_42602_2020')
no2 = orgininal_no2.loc[orgininal_no2['State Name'] == 'California']
no2 = no2.loc[:,['Date Local','Arithmetic Mean' ,'1st Max Value', 'AQI']]
regr = linear_model.LinearRegression()
regr.fit(no2[['Arithmetic Mean' ,'1st Max Value']], no2.loc[:,'AQI'])
print(np.corrcoef(no2.loc[:,'1st Max Value'], no2.loc[:,'AQI'])[0][1])
print(np.corrcoef(no2.loc[:,'Arithmetic Mean'], no2.loc[:,'AQI'])[0][1])
print(regr.intercept_)
regr.coef_

0.9993527271392962
0.9233244638388015
-0.25824907307965006


array([0.00624619, 0.93756385])

In [99]:
orgininal_co = epa_data.get('daily_42101_2020')
co = orgininal_co.loc[orgininal_co['State Name'] == 'California']
co = co.loc[:,['Date Local','Arithmetic Mean' ,'1st Max Value', 'AQI']].dropna()
regr = linear_model.LinearRegression()
regr.fit(co[['Arithmetic Mean' ,'1st Max Value']], co.loc[:,'AQI'])
print(np.corrcoef(co.loc[:,'1st Max Value'], co.loc[:,'AQI'])[0][1])
print(np.corrcoef(co.loc[:,'Arithmetic Mean'], co.loc[:,'AQI'])[0][1])
print(regr.intercept_)
regr.coef_

-0.1498405278407109
0.9965504948770234
0.9400152629775486


array([ 0.23699921, 11.46867252])

In [100]:
orgininal_ozone = epa_data.get('daily_44201_2020')
ozone = orgininal_ozone.loc[orgininal_ozone['State Name'] == 'California']
ozone = ozone.loc[:,['Date Local','Arithmetic Mean' ,'1st Max Value', 'AQI']].dropna()
regr = linear_model.LinearRegression(fit_intercept = True)
regr.fit(ozone[['Arithmetic Mean' ,'1st Max Value']], ozone.loc[:,'AQI'])
print(np.corrcoef(ozone.loc[:,'1st Max Value'], ozone.loc[:,'AQI'])[0][1])
print(np.corrcoef(ozone.loc[:,'Arithmetic Mean'], ozone.loc[:,'AQI'])[0][1])
print(regr.intercept_)
regr.coef_

-27.084710788807357
0.938539887180763
0.7978867718732714


array([-316.03147476, 1913.87297034])

In [138]:
epa_data.get('epa_data_CA_merged')

,Unnamed: 0,State Name,county Name,Month,Day,AQI,Category,Defining Site,Latitude,Longitude
0,0,California,Alameda,1,1,53,Moderate,06-001-0009,37.743065,-122.169935
1,1,California,Alameda,1,2,43,Good,06-001-0013,37.864767,-122.302741
2,2,California,Alameda,1,3,74,Moderate,06-001-0013,37.864767,-122.302741
3,3,California,Alameda,1,4,45,Good,06-001-0007,37.687526,-121.784217
4,4,California,Alameda,1,5,33,Good,06-001-0007,37.687526,-121.784217
...,...,...,...,...,...,...,...,...,...,...
19220,19220,California,Yolo,12,27,20,Good,06-113-0004,38.534450,-121.773400
19221,19221,California,Yolo,12,28,33,Good,06-113-0004,38.534450,-121.773400
19222,19222,California,Yolo,12,29,28,Good,06-113-0004,38.534450,-121.773400
19223,19223,California,Yolo,12,30,39,Good,06-113-0004,38.534450,-121.773400


In [150]:
epa_data.get('daily_42101_2020')

AQI  Arithmetic Mean  1st Max Value
County Name Date Local                                      
Alameda     2020-01-01   8.0         0.519155          0.860
            2020-01-02  10.6         0.648320          1.060
            2020-01-03  12.8         0.814800          1.230
            2020-01-04   9.8         0.565381          0.960
            2020-01-05   9.2         0.481263          0.880
...                      ...              ...            ...
Stanislaus  2020-12-27  13.0         0.955646          1.211
            2020-12-28  13.0         0.971437          1.147
            2020-12-29  18.0         1.116709          1.787
            2020-12-30  19.0         1.142562          1.536
            2020-12-31  13.0         0.948271          1.092

[8995 rows x 3 columns]

In [147]:
epa_data_CA_merged = epa_data.get('epa_data_CA_merged')
tuples = list(zip(epa_data_CA_merged['county Name'].to_list(), orgininal_no2['Daily'].to_list(),epa_data_CA_merged['Latitude'].to_list()))

In [149]:
pd.DataFrame(tuples)

,0,1,2
0,Alameda,-122.169935,37.743065
1,Alameda,-122.302741,37.864767
2,Alameda,-122.302741,37.864767
3,Alameda,-121.784217,37.687526
4,Alameda,-121.784217,37.687526
...,...,...,...
19220,Yolo,-121.773400,38.534450
19221,Yolo,-121.773400,38.534450
19222,Yolo,-121.773400,38.534450
19223,Yolo,-121.773400,38.534450
